# How to run the program
To run this notebook, click kernel and run all. This should install the dependancies, perform the blackbox tests and start the GUI.


# Dependacies
 
Installing required libaries using pip.

In [1]:
#Un-comment if installing is required


!pip install opencv-python
!pip install mediapipe
!pip install keyboard


# Import Required Libaries
Setting up the imports for the project and populating an array with the headers for the data about to be captured and a key press boolean for recording keyboard inputs for exporting the array to a csv file.

In [2]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import mediapipe as mp
import time
import os
import sys
import keyboard
import time
import keras
from keras.models import Sequential
import numpy as np
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import csv

csv = 'X_0, Y_0, X_1, Y_1, X_2, Y_2, X_3, Y_3, X_4, Y_4, X_5, Y_5, X_6, Y_6, X_7, Y_7, X_8, Y_8, X_9, Y_9, X_10, Y_10, X_11, Y_11, X_12, Y_12, X_13, Y_13, X_14, Y_14, X_15, Y_15, X_16, Y_16, X_17, Y_17, X_18, Y_18, X_19, Y_19, X_20, Y_20, Label\n'

reconstructed_model = keras.models.load_model("CNN_Hand_Model", compile = True)

keydown = False

# Hand Detector Class

Creating a hand detector class for reusability. Handles using OpenCV model to location hand landmarks and return position information for later use.

In [3]:
#Creating a hand detector class to allow this it be used for multiple projects

class handDetector():
    
    def __init__(self, mode=False, Hands =2, Complexity=1, detectionConf=0.5, trackConf=0.5):
        
        self.mode = mode
        self.Hands = Hands
        self.Complexity = Complexity
        self.detectionConf = detectionConf
        self.trackConf = trackConf
        
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.Hands, self.Complexity, self.detectionConf, self.trackConf)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True):
        
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, 
                                               self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handNo=0, draw=True):
        
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            
            for id, lm in enumerate(myHand.landmark):

                    h ,w, c = img.shape
                    cx, cy = int(lm.x*w), int(lm.y*h)
                    lmList.append([id, cx, cy])
                    if draw:
                        cv2.circle(img, (cx,cy), 15, (255, 0, 255), cv2.FILLED)
                    
        return lmList


# Model Prediction

Function to handle data formating returning the model prediction in main loop.

In [4]:

def modelprediction(lmList):
    buffer = [[]]
    i = 0
    for lm in lmList:
        buffer[0].insert(i, float(lm[1]))
        buffer[0].insert(i+1, float(lm[2]))
        i += 2
    
    predictions = reconstructed_model.predict(buffer)
    return predictions


# Logical Finger Counter

As basic hand signs are use, logic can be used to count fingers with very high accuracy. By doing this the model can be checked on all inputs to ensure the classifcation is correct.

In [5]:
def numberoffingers(lmList, tipIds):
    fingers = []

    # thumb
    if lmList[tipIds[0]][1] > lmList[tipIds[0]-1][1]:
        fingers.append(1)
    else:
        fingers.append(0)

    # 4 fingers
    for id in range(1,5):
        if lmList[tipIds[id]][2] < lmList[tipIds[id]-2][2]:
            fingers.append(1)
        else:
            fingers.append(0)

    totalFingers = fingers.count(1)

    return totalFingers

# Model Prediction
This function is used for the blackbox testing.

In [6]:
def modelpredict(lmList):
    predict = modelprediction(lmList)
    predict = np.argmax(predict)
    return predict

# Main loop function
Function used for drawing the new image to GUI along with all relevant label updates.

In [7]:
def show_frames():
    
    predict = []
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    tipIds = [ 4, 8, 12, 16, 20]
    
    if len(lmList) !=0:
        totalFingers = numberoffingers(lmList, tipIds)
        predict = modelpredict(lmList)
        num_fingers.set(str(predict))

        if predict == totalFingers:
            #cv2.putText(img, str('Correct'),(45,375),cv2.FONT_HERSHEY_PLAIN,10,(0,255,0),25)
            correct.set('Correct')
        else:
            #cv2.putText(img, str('Incorrect'),(45,375),cv2.FONT_HERSHEY_PLAIN,10,(255,0,0),25)
            correct.set('Incorrect')
      
    # Get the latest frame and convert into Image
    cv2image= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    # Convert image to PhotoImage
    imgtk = ImageTk.PhotoImage(image = img)
    cam_label.imgtk = imgtk
    cam_label.configure(image=imgtk)
    # Repeat after an interval to capture continiously
    cam_label.after(20, show_frames)


# Black Box Tests
Tests used to known data recording that represents the correct output for each number of held up fingers, this allows testing of both the model and logic function ability to correctly classify.

In [8]:
def test_5_finger():
    lmList = [[0, 448, 407], [1, 489, 344], [2, 504, 270], [3, 507, 208], [4, 513, 158], [5, 411, 216], [6, 397, 145], [7, 383, 104], [8, 369, 70], [9, 369, 233], [10, 339, 164], [11, 317, 125], [12, 298, 95], [13, 338, 263], [14, 299, 206], [15, 273, 172], [16, 253, 143], [17, 317, 305], [18, 276, 274], [19, 249, 253], [20, 226, 230]]
    tipIds = [ 4, 8, 12, 16, 20]
    assert modelpredict(lmList) == 5
    assert numberoffingers(lmList, tipIds) == 5
    print("Test 5 Fingers Complete.")
    
test_5_finger()

Test 5 Fingers Complete.


In [9]:
def test_4_finger():
    lmList = [[0, 292, 437], [1, 353, 393], [2, 381, 326], [3, 398, 267], [4, 426, 231], [5, 321, 252], [6, 325, 183], [7, 325, 141], [8, 324, 103], [9, 279, 256], [10, 276, 184], [11, 273, 140], [12, 270, 102], [13, 244, 280], [14, 209, 218], [15, 187, 181], [16, 171, 145], [17, 216, 319], [18, 194, 291], [19, 204, 314], [20, 220, 336]]
    tipIds = [ 4, 8, 12, 16, 20]
    assert modelpredict(lmList) == 4
    assert numberoffingers(lmList, tipIds) == 4
    print("Test 4 Fingers Complete.")
    
test_4_finger()

Test 4 Fingers Complete.


In [10]:
def test_3_finger():
    lmList = [[0, 290, 448], [1, 350, 405], [2, 384, 339], [3, 420, 292], [4, 459, 267], [5, 327, 259], [6, 344, 187], [7, 353, 141], [8, 357, 101], [9, 281, 260], [10, 272, 183], [11, 263, 130], [12, 254, 86], [13, 244, 284], [14, 209, 250], [15, 227, 305], [16, 245, 341], [17, 213, 320], [18, 186, 307], [19, 204, 342], [20, 226, 366]]
    tipIds = [ 4, 8, 12, 16, 20]
    assert modelpredict(lmList) == 3
    assert numberoffingers(lmList, tipIds) == 3
    print("Test 3 Fingers Complete.")
    
test_3_finger()

Test 3 Fingers Complete.


In [11]:
def test_2_finger():
    lmList = [[0, 330, 438], [1, 392, 394], [2, 435, 324], [3, 474, 269], [4, 519, 239], [5, 362, 234], [6, 370, 158], [7, 369, 106], [8, 368, 65], [9, 313, 245], [10, 281, 205], [11, 288, 268], [12, 303, 309], [13, 270, 267], [14, 236, 244], [15, 256, 300], [16, 276, 326], [17, 236, 297], [18, 207, 277], [19, 227, 315], [20, 251, 333]]
    tipIds = [ 4, 8, 12, 16, 20]
    assert modelpredict(lmList) == 2
    assert numberoffingers(lmList, tipIds) == 2
    print("Test 2 Fingers Complete.")
    
test_2_finger()

Test 2 Fingers Complete.


In [12]:
def test_1_finger():
    lmList = [[0, 353, 367], [1, 403, 321], [2, 443, 261], [3, 489, 212], [4, 540, 199], [5, 388, 174], [6, 355, 171], [7, 366, 222], [8, 388, 214], [9, 338, 186], [10, 311, 174], [11, 328, 235], [12, 347, 225], [13, 294, 202], [14, 268, 183], [15, 287, 238], [16, 312, 232], [17, 257, 219], [18, 236, 207], [19, 251, 242], [20, 273, 245]]
    tipIds = [ 4, 8, 12, 16, 20]
    assert modelpredict(lmList) == 1
    assert numberoffingers(lmList, tipIds) == 1
    print("Test 1 Fingers Complete.")
    
test_1_finger()

Test 1 Fingers Complete.


In [13]:
def test_0_finger():
    lmList = [[0, 447, 324], [1, 489, 281], [2, 518, 214], [3, 494, 174], [4, 447, 181], [5, 451, 160], [6, 446, 122], [7, 463, 174], [8, 467, 188], [9, 410, 170], [10, 400, 132], [11, 421, 187], [12, 421, 196], [13, 373, 190], [14, 360, 154], [15, 386, 201], [16, 389, 210], [17, 334, 216], [18, 330, 189], [19, 352, 219], [20, 357, 230]]
    tipIds = [ 4, 8, 12, 16, 20]
    assert modelpredict(lmList) == 0
    assert numberoffingers(lmList, tipIds) == 0
    print("Test 0 Fingers Complete.")
    
test_0_finger()

Test 0 Fingers Complete.


# Main Method
Initilised the camera source, creating the GUI variables and running GUI loop for the program to work.

In [14]:
wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
    
detector = handDetector(detectionConf=0.75)

# Create an instance of TKinter Window or frame
#gui creation
win = Tk()

# Set the size of the window, stop resize, rename title
win.geometry("850x480")
win.resizable(False, False)
win.title('AI and Robotics - John Robson')

#Declaring changing variables
num_fingers = StringVar()
correct = StringVar()

# Create a Label to capture the Video frames and labal changes
finger_title = Label(win, text="Number of fingers predicted")
finger_title.place(x=660, y=10)

finger_label = Label(win, textvariable=num_fingers)
finger_label.place(x=720, y=30)

correct_title = Label(win, text="The classification was:")
correct_title.place(x=720, y=70, anchor=CENTER)

correct_label = Label(win, textvariable=correct) 
correct_label.place(x=720, y=100, anchor=CENTER)

cam_label = Label(win)
cam_label.grid(row=0, column=0)

# Define function to show frame
#main

show_frames()
win.mainloop()